<a href="https://colab.research.google.com/github/DOOMNOVA/Semantic-Textual-Similarity-STS-/blob/main/Finetuning_Part_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"This file included finetuning a new model using the pseudo-labelled dataset"

"install relevant libraries"
# run cell to install packages
!pip install datasets
!pip install transformers[torch]
#!pip install accelerate -U
#!pip install seqeval
#!pip install evaluate
#!pip install pipreqs
!pip install -U "huggingface_hub[cli]"
#!pip install scikit-learn seaborn matplotlib
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 901.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━

In [3]:
import transformers
import torch
print(transformers.__version__)
print(torch.__version__)
import sys
print(sys.version)


4.38.2
2.2.1+cu121
3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


In [2]:
"login to hugging face to upload model to hub later - uncomment code if needed"
from huggingface_hub import notebook_login
notebook_login(True)


In [3]:
"import relevant libraries"
import pandas as pd
import torch
from google.colab import files
import io
from sentence_transformers import SentenceTransformer, util



"load the train dataset with the new pseudolabels"
uploaded = files.upload()



Saving DataNeuron_Text_Similarity_pseudo.csv to DataNeuron_Text_Similarity_pseudo.csv


In [4]:
# Define a function to calculate cosine similarity for a pair of sentences
def calculate_cosine_similarity(text1, text2):
    embedding1 = model.encode(text1, convert_to_tensor=True)
    embedding2 = model.encode(text2, convert_to_tensor=True)
    similarity_sco = util.cos_sim(embedding1, embedding2)
    #normalize to the range 0,1
    similarity_score_normalized = (similarity_sco.item() + 1) / 2
    print(similarity_score_normalized)
    return similarity_score_normalized


In [5]:
"Load the new pseudo-labelled dataset"
df_pseudo_train = pd.read_csv(io.BytesIO(uploaded['DataNeuron_Text_Similarity_pseudo.csv']))
df_pseudo_train = df_pseudo_train.drop(columns=['Unnamed: 0'])

In [6]:
df_pseudo_train

,text1,text2,cosine_similarity
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...,0.517517
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...,0.592068
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...,0.539645
3,hearts of oak 3-2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...,0.445197
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...,0.527641
...,...,...,...
2995,uk directors guild nominees named martin scors...,steel firm to cut 45 000 jobs mittal steel ...,0.530638
2996,u2 to play at grammy awards show irish rock ba...,israel looks to us for bank chief israel has a...,0.494892
2997,pountney handed ban and fine northampton coach...,india and iran in gas export deal india has si...,0.457765
2998,belle named best scottish band belle & sebas...,mido makes third apology ahmed mido hossam h...,0.478333


In [7]:
"Transform the data into a suitable format for finetuning"
from sentence_transformers import losses,InputExample
from datasets import load_dataset

train_examples = []

for idx, row in df_pseudo_train.iterrows():
  text1 = row['text1']
  text2 = row['text2']
  similarity_score = row['cosine_similarity']

  example = InputExample(texts=[text1,text2],label= similarity_score)
  train_examples.append(example)












In [17]:
"Fine tune the pretrained model using the pseudo labelled train dataset"
from torch.utils.data import DataLoader


model_a = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
train_loss = losses.CosineSimilarityLoss(model=model_a)
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

In [18]:
model_a.fit(train_objectives=[(train_dataloader, train_loss)], epochs=5)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/188 [00:00<?, ?it/s]

Iteration:   0%|          | 0/188 [00:00<?, ?it/s]

Iteration:   0%|          | 0/188 [00:00<?, ?it/s]

Iteration:   0%|          | 0/188 [00:00<?, ?it/s]

Iteration:   0%|          | 0/188 [00:00<?, ?it/s]

In [19]:
"Save the fine tuned model to HuggingFace modelhub"
model_a.save_to_hub(
    "Finetuned_STS_part_A_final",
    exist_ok=True,
    )

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

'https://huggingface.co/doomnova/Finetuned_STS_part_A_final/commit/b67826b86f4175ce3f08cd26015e5ebbe6405a11'

In [ ]:
"helper function for deployment"
def predict_fn(data,model):

    #error handling
    if "text1" not in data or "text2" not in data:
        raise ValueError("Request body must contain keys 'text1' and 'text2'")

    # Check if the values corresponding to the keys are strings
    if not isinstance(data["text1"], str) or not isinstance(data["text2"], str):
        raise ValueError("Values for 'text1' and 'text2' must be strings")

    # convert the texts to embeddings
    embedding1 = model.encode(data["text1"], convert_to_tensor=True)
    embedding2 = model.encode(data["text2"], convert_to_tensor=True)

    #calculate the cosine similarity
    similarity_sco = util.cos_sim(embedding1, embedding2)
    #normalize cosine similarity to the range (0,1)
    similarity_score_normalized = (similarity_sco.item() + 1) / 2
    return {"similarity score" : similarity_score_normalized}

#load model
#def model_fn(model_dir):
 #   model = SentenceTransformer(model_dir)
 #   return model
